In [3]:
# Install Spark
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-16 02:29:52--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.01MB/s    in 0.2s    

2021-12-16 02:29:53 (6.01 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter PW')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://challenge-game-reviews.ctg85syqs9uc.us-east-1.rds.amazonaws.com:5432/amazon_vine_analysis"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter PW··········


In [7]:
# Read in vine_table from RDS instance
vine_df = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [8]:
# 1 - Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20.
vine_vote20_df = vine_df.filter("total_votes>=20")
vine_vote20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [9]:
# Confirm data types
vine_vote20_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [10]:
# 2A - Create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes
import pyspark.sql.functions as F
vine_vote_perc_df = vine_vote20_df.withColumn("Helpful_Percentage", (F.col("helpful_votes") / F.col("total_votes")*100))
vine_vote_perc_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N| 61.76470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|              60.0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|              84.0|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y| 45.16129032258064|
|R2FJ94555FZH32|          2|           55|         60|   N|                N| 91.66666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y| 78.46153846153847|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N| 86.11111111111111|
| R6KTC1OPIOIIG|          2|  

In [11]:
# 2B - Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
vine_vote_perc_updated_df = vine_vote_perc_df.filter("Helpful_Percentage>=50.0")
vine_vote_perc_updated_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N| 61.76470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|              60.0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|              84.0|
|R2FJ94555FZH32|          2|           55|         60|   N|                N| 91.66666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y| 78.46153846153847|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N| 86.11111111111111|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y| 55.88235294117647|
|R36O341WWXXKNP|          5|  

In [12]:
# 3 - Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_reviews_y_df = vine_vote_perc_updated_df.filter(vine_vote_perc_updated_df.vine == 'Y')
vine_reviews_y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N| 88.88888888888889|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|             100.0|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N| 79.16666666666666|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N| 90.19607843137256|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N| 84.61538461538461|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N| 76.92307692307693|
|R3N4QAXTQCA1ED|          5|           18|         20|   Y|                N|              90.0|
|R1JRR530H4COA2|          5|  

In [13]:
# 4 - Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_reviews_n_df = vine_vote_perc_updated_df.filter(vine_vote_perc_updated_df.vine == 'N')
vine_reviews_n_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|Helpful_Percentage|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N| 61.76470588235294|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|              60.0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|              84.0|
|R2FJ94555FZH32|          2|           55|         60|   N|                N| 91.66666666666666|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y| 78.46153846153847|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N| 86.11111111111111|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y| 55.88235294117647|
|R36O341WWXXKNP|          5|  

In [14]:
# 5A - Determine the total number of reviews for the two types of review (paid vs unpaid).
#print("Total Paid Reviews & Column Count: ", (vine_reviews_y_df.count(), len(vine_reviews_y_df.columns)))
#print("Total Non-Paid Reviews & Column Count: ", (vine_reviews_n_df.count(), len(vine_reviews_n_df.columns)))
total_paid_reviews = vine_reviews_y_df.count()
print("Total Paid Reviews: ",total_paid_reviews)
total_non_paid_reviews = vine_reviews_n_df.count()
print("Total Non-Paid Reviews: ",total_non_paid_reviews)

Total Paid Reviews:  94
Total Non-Paid Reviews:  40471


In [16]:
# 5B - Determine the number of 5-star reviews for the two types of review (paid vs unpaid).
five_star_paid_reviews = vine_reviews_y_df.filter(vine_reviews_n_df.star_rating == 5).count()
print("Number of Paid 5-Star Reviews: ", vine_reviews_y_df.filter(vine_reviews_n_df.star_rating == 5).count())
#print(five_star_paid_reviews)
five_star_non_paid_reviews = vine_reviews_n_df.filter(vine_reviews_n_df.star_rating == 5).count()
print("Number of Non Paid 5-Star Reviews: ", vine_reviews_n_df.filter(vine_reviews_n_df.star_rating == 5).count())
#print(five_star_non_paid_reviews)

Number of Paid 5-Star Reviews:  48
Number of Non Paid 5-Star Reviews:  15663


In [20]:
# 5C - Determine the percentage of 5-star reviews for the two types of review (paid vs unpaid).
paid_5_star_rate = ((five_star_paid_reviews / total_paid_reviews)*100)
print("Paid 5 Star Rate: ",paid_5_star_rate)
non_paid_5_star_rate = ((five_star_non_paid_reviews / total_non_paid_reviews)*100)
print("Non Paid 5 Star Rate: ",non_paid_5_star_rate)

Paid 5 Star Rate:  51.06382978723404
Non Paid 5 Star Rate:  38.701786464381904


In [37]:
results_df = spark.createDataFrame(
    [
     ("Paid/Vine", total_paid_reviews, five_star_paid_reviews, paid_5_star_rate),
     ("Unpaid/non-Vine", total_non_paid_reviews, five_star_non_paid_reviews, non_paid_5_star_rate),
    ],
    ["Review Type", "Total Reviews", "Total 5-Star Reviews", "5-Star Rate"]
)
results_df.show()

+---------------+-------------+--------------------+------------------+
|    Review Type|Total Reviews|Total 5-Star Reviews|       5-Star Rate|
+---------------+-------------+--------------------+------------------+
|      Paid/Vine|           94|                  48| 51.06382978723404|
|Unpaid/non-Vine|        40471|               15663|38.701786464381904|
+---------------+-------------+--------------------+------------------+



In [38]:
results_df = results_df.withColumn('Total Reviews (#)', F.format_number('Total Reviews', 0))
results_df = results_df.withColumn('Total 5-Star Reviews (#)', F.format_number('Total 5-Star Reviews', 0))
results_df = results_df.withColumn('5-Star Rate (%)', F.format_number('5-Star Rate', 2))
results_df = results_df.drop("Total Reviews", "Total 5-Star Reviews", "5-Star Rate")
results_df.show()

+---------------+-----------------+------------------------+---------------+
|    Review Type|Total Reviews (#)|Total 5-Star Reviews (#)|5-Star Rate (%)|
+---------------+-----------------+------------------------+---------------+
|      Paid/Vine|               94|                      48|          51.06|
|Unpaid/non-Vine|           40,471|                  15,663|          38.70|
+---------------+-----------------+------------------------+---------------+

